In [1]:
import sys
sys.path.insert(0, "..\scripts")

In [2]:
import transform_db
from transform_db import load_clean
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option("display.max_columns",None)

In [3]:
load = load_clean("C:\\Users\\alexp\\src\\NBA_Models\\sqlite\\db\\nba_data.db")
agg_boxscores = load.boxscore_matchups()

In [4]:
display(agg_boxscores.tail())

,GAME_ID,TEAM_ID_home,TEAM_ID_away,SEASON_home,GAME_DATE_home,TEAM_NAME_home,TEAM_NAME_away,MATCHUP_home,TEAM_ABBREVIATION_home,HOME_GAME_home,WL_home,FG2M_home,FG2A_home,FG3M_home,FG3A_home,FTM_home,FTA_home,OREB_home,DREB_home,REB_home,AST_home,STL_home,BLK_home,TOV_home,PF_home,PTS_home,PLUS_MINUS_home,E_OFF_RATING_home,OFF_RATING_home,E_DEF_RATING_home,DEF_RATING_home,E_NET_RATING_home,NET_RATING_home,POSS_home,PIE_home,PTS_2PT_MR_home,PTS_FB_home,PTS_OFF_TOV_home,PTS_PAINT_home,AST_2PM_home,AST_3PM_home,UAST_2PM_home,UAST_3PM_home,TEAM_ABBREVIATION_away,HOME_GAME_away,WL_away,FG2M_away,FG2A_away,FG3M_away,FG3A_away,FTM_away,FTA_away,OREB_away,DREB_away,REB_away,AST_away,STL_away,BLK_away,TOV_away,PF_away,PTS_away,PLUS_MINUS_away,E_OFF_RATING_away,OFF_RATING_away,E_DEF_RATING_away,DEF_RATING_away,E_NET_RATING_away,NET_RATING_away,POSS_away,PIE_away,PTS_2PT_MR_away,PTS_FB_away,PTS_OFF_TOV_away,PTS_PAINT_away,AST_2PM_away,AST_3PM_away,UAST_2PM_away,UAST_3PM_away
11855,0022300482,1610612739,1610612764,2023-24,2024-01-05 00:00:00,Cleveland Cavaliers,Washington Wizards,CLE vs. WAS,CLE,1,1,25,44,16,33,16,23,12,40,52,31,9,1,24,19,114,24,115.0,117.5,89.5,91.8,25.5,25.7,97.0,0.643,2,12,18,47,16,15,9,1,WAS,0,0,26,54,9,30,11,15,7,24,31,19,12,8,17,19,90,-24,89.5,91.8,115.0,117.5,-25.5,-25.7,98.0,0.357,1,15,28,50,10,8,15,0
11856,0022300481,1610612751,1610612760,2023-24,2024-01-05 00:00:00,Brooklyn Nets,Oklahoma City Thunder,BKN vs. OKC,BKN,1,1,27,52,14,39,28,33,20,31,51,25,9,3,15,24,124,9,123.4,124.0,114.4,113.9,9.0,10.1,100.0,0.513,5,13,11,47,11,13,15,0,OKC,0,0,36,53,9,31,16,24,7,29,36,29,8,5,13,26,115,-9,114.4,113.9,123.4,124.0,-9.0,-10.1,101.0,0.487,8,12,20,64,20,8,15,0
11857,0022300483,1610612755,1610612752,2023-24,2024-01-05 00:00:00,Philadelphia 76ers,New York Knicks,PHI vs. NYK,PHI,1,0,27,58,6,23,20,27,8,30,38,22,8,9,14,18,92,-36,93.0,92.0,124.3,128.0,-31.2,-36.0,100.0,0.366,8,16,11,46,16,6,10,0,NYK,0,1,25,48,18,41,24,25,10,39,49,27,7,7,13,22,128,36,124.3,128.0,93.0,92.0,31.2,36.0,100.0,0.634,6,33,14,44,13,14,12,3
11858,0022300484,1610612741,1610612766,2023-24,2024-01-05 00:00:00,Chicago Bulls,Charlotte Hornets,CHI vs. CHA,CHI,1,1,30,46,10,28,14,17,4,37,41,29,7,6,15,17,104,13,112.5,111.8,95.5,97.8,16.9,14.0,93.0,0.633,11,24,19,48,18,10,11,0,CHA,0,0,26,55,10,34,9,12,12,30,42,20,7,1,13,20,91,-13,95.5,97.8,112.5,111.8,-16.9,-14.0,93.0,0.367,16,10,16,36,10,9,15,1
11859,0022300486,1610612740,1610612746,2023-24,2024-01-05 00:00:00,New Orleans Pelicans,LA Clippers,NOP vs. LAC,NOP,1,0,21,63,12,29,17,23,14,29,43,22,9,5,9,17,95,-16,97.8,101.1,115.9,118.1,-18.0,-17.0,94.0,0.391,3,11,17,38,12,9,8,3,LAC,0,1,25,46,15,36,16,20,10,40,50,28,6,7,15,18,111,16,115.9,118.1,97.8,101.1,18.0,17.0,94.0,0.609,9,12,7,39,16,12,9,3


### Data Preparation

In [5]:
column_filter = ['GAME_DATE_home','HOME_GAME_home','TEAM_ABBREVIATION_home','WL_home', 'FG2M_home',
       'FG2A_home', 'FG3M_home', 'FG3A_home', 'FTM_home', 'FTA_home',
       'OREB_home', 'DREB_home', 'REB_home', 'AST_home', 'STL_home',
       'BLK_home', 'TOV_home', 'PF_home', 'PTS_home', 'PLUS_MINUS_home',
       'E_OFF_RATING_home', 'OFF_RATING_home', 'E_DEF_RATING_home',
       'DEF_RATING_home', 'E_NET_RATING_home', 'NET_RATING_home', 'POSS_home',
       'PIE_home', 'PTS_2PT_MR_home', 'PTS_FB_home', 'PTS_OFF_TOV_home',
       'PTS_PAINT_home', 'AST_2PM_home', 'AST_3PM_home', 'UAST_2PM_home',
       'UAST_3PM_home', 'HOME_GAME_away','TEAM_ABBREVIATION_away', 'WL_away',
       'FG2M_away', 'FG2A_away', 'FG3M_away', 'FG3A_away', 'FTM_away',
       'FTA_away', 'OREB_away', 'DREB_away', 'REB_away', 'AST_away',
       'STL_away', 'BLK_away', 'TOV_away', 'PF_away', 'PTS_away',
       'PLUS_MINUS_away', 'E_OFF_RATING_away', 'OFF_RATING_away',
       'E_DEF_RATING_away', 'DEF_RATING_away', 'E_NET_RATING_away',
       'NET_RATING_away', 'POSS_away', 'PIE_away', 'PTS_2PT_MR_away',
       'PTS_FB_away', 'PTS_OFF_TOV_away', 'PTS_PAINT_away', 'AST_2PM_away',
       'AST_3PM_away', 'UAST_2PM_away', 'UAST_3PM_away']


In [42]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(agg_boxscores['TEAM_ABBREVIATION_home'])

TEAM_ID_lst = set(agg_boxscores['TEAM_ID_home'])

team_list_dfs = []
X = []
y = []

window_size = 10

for id in TEAM_ID_lst:
    gsw_df = agg_boxscores.loc[((agg_boxscores['TEAM_ID_home'] == id) | (agg_boxscores['TEAM_ID_away'] == id)) & (agg_boxscores['SEASON_home'] != '2023-24')]
    DATA_DROP = gsw_df[column_filter]

    DATA_DROP['NAME_ENC_home'] = encoder.transform(DATA_DROP['TEAM_ABBREVIATION_home'])
    DATA_DROP['NAME_ENC_away'] = encoder.transform(DATA_DROP['TEAM_ABBREVIATION_away'])

    column_filter2 = ['HOME_GAME_home','NAME_ENC_home', 'FG2M_home',
       'FG2A_home', 'FG3M_home', 'FG3A_home', 'FTM_home', 'FTA_home',
       'OREB_home', 'DREB_home', 'REB_home', 'AST_home', 'STL_home',
       'BLK_home', 'TOV_home', 'PF_home', 'PTS_home', 'PLUS_MINUS_home',
       'E_OFF_RATING_home', 'OFF_RATING_home', 'E_DEF_RATING_home',
       'DEF_RATING_home', 'E_NET_RATING_home', 'NET_RATING_home', 'POSS_home',
       'PIE_home', 'PTS_2PT_MR_home', 'PTS_FB_home', 'PTS_OFF_TOV_home',
       'PTS_PAINT_home', 'AST_2PM_home', 'AST_3PM_home', 'UAST_2PM_home',
       'UAST_3PM_home', 'HOME_GAME_away','NAME_ENC_away',
       'FG2M_away', 'FG2A_away', 'FG3M_away', 'FG3A_away', 'FTM_away',
       'FTA_away', 'OREB_away', 'DREB_away', 'REB_away', 'AST_away',
       'STL_away', 'BLK_away', 'TOV_away', 'PF_away', 'PTS_away',
       'PLUS_MINUS_away', 'E_OFF_RATING_away', 'OFF_RATING_away',
       'E_DEF_RATING_away', 'DEF_RATING_away', 'E_NET_RATING_away',
       'NET_RATING_away', 'POSS_away', 'PIE_away', 'PTS_2PT_MR_away',
       'PTS_FB_away', 'PTS_OFF_TOV_away', 'PTS_PAINT_away', 'AST_2PM_away',
       'AST_3PM_away', 'UAST_2PM_away', 'UAST_3PM_away'] 
    DATA = DATA_DROP[column_filter2]
    LABEL = DATA_DROP['WL_home']
    DATA_np = DATA.to_numpy()
    LABEL_np = LABEL.to_numpy()
    for i in range(len(DATA_np)-window_size):        
      row = [a for a in DATA_np[i:i+window_size]]
      X.append(row)
      label = LABEL_np[i+window_size]
      y.append(label)

      X = np.array(X)
      y = np.array(y)


C:\Users\alexp\AppData\Local\Temp\ipykernel_3380\3634417163.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA_DROP['NAME_ENC_home'] = encoder.transform(DATA_DROP['TEAM_ABBREVIATION_home'])
C:\Users\alexp\AppData\Local\Temp\ipykernel_3380\3634417163.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA_DROP['NAME_ENC_away'] = encoder.transform(DATA_DROP['TEAM_ABBREVIATION_away'])
C:\Users\alexp\AppData\Local\Temp\ipykernel_3380\3634417163.py:18: SettingWithCopyWarning: 
A value is trying to be se

In [103]:
DATA_DROP = agg_boxscores[column_filter]

DATA_DROP['NAME_ENC_home'] = encoder.transform(DATA_DROP['TEAM_ABBREVIATION_home'])
DATA_DROP['NAME_ENC_away'] = encoder.transform(DATA_DROP['TEAM_ABBREVIATION_away'])

X2 = []
y2 = []

column_filter2 = ['HOME_GAME_home','NAME_ENC_home', 'FG2M_home',
   'FG2A_home', 'FG3M_home', 'FG3A_home', 'FTM_home', 'FTA_home',
   'OREB_home', 'DREB_home', 'REB_home', 'AST_home', 'STL_home',
   'BLK_home', 'TOV_home', 'PF_home', 'PTS_home', 'PLUS_MINUS_home',
   'E_OFF_RATING_home', 'OFF_RATING_home', 'E_DEF_RATING_home',
   'DEF_RATING_home', 'E_NET_RATING_home', 'NET_RATING_home', 'POSS_home',
   'PIE_home', 'PTS_2PT_MR_home', 'PTS_FB_home', 'PTS_OFF_TOV_home',
   'PTS_PAINT_home', 'AST_2PM_home', 'AST_3PM_home', 'UAST_2PM_home',
   'UAST_3PM_home', 'HOME_GAME_away','NAME_ENC_away',
   'FG2M_away', 'FG2A_away', 'FG3M_away', 'FG3A_away', 'FTM_away',
   'FTA_away', 'OREB_away', 'DREB_away', 'REB_away', 'AST_away',
   'STL_away', 'BLK_away', 'TOV_away', 'PF_away', 'PTS_away',
   'PLUS_MINUS_away', 'E_OFF_RATING_away', 'OFF_RATING_away',
   'E_DEF_RATING_away', 'DEF_RATING_away', 'E_NET_RATING_away',
   'NET_RATING_away', 'POSS_away', 'PIE_away', 'PTS_2PT_MR_away',
   'PTS_FB_away', 'PTS_OFF_TOV_away', 'PTS_PAINT_away', 'AST_2PM_away',
   'AST_3PM_away', 'UAST_2PM_away', 'UAST_3PM_away'] 

DATA = DATA_DROP[column_filter2]
LABEL = DATA_DROP['WL_home']
DATA_np = DATA.to_numpy()
LABEL_np = LABEL.to_numpy()
for i in range(len(DATA_np)-window_size):        
  row = [a for a in DATA_np[i:i+window_size]]
  X2.append(row)
  label = LABEL_np[i+window_size]
  y2.append(label)

X2 = np.array(X2)
y2 = np.array(y2)

C:\Users\alexp\AppData\Local\Temp\ipykernel_3380\2553213591.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA_DROP['NAME_ENC_home'] = encoder.transform(DATA_DROP['TEAM_ABBREVIATION_home'])
C:\Users\alexp\AppData\Local\Temp\ipykernel_3380\2553213591.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA_DROP['NAME_ENC_away'] = encoder.transform(DATA_DROP['TEAM_ABBREVIATION_away'])


In [104]:
X2.shape

(11830, 30, 68)

In [106]:
X_train, y_train = X2[:8000], y2[:8000]
X_val, y_val = X2[8000:10000], y2[8000:10000]
X_test, y_test = X2[10000:], y2[10000:]
X_train.shape, y_train.shape

((8000, 30, 68), (8000,))

In [133]:
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError, Accuracy
from tensorflow.keras.optimizers import Adam

model1 = Sequential()

model1.add(InputLayer((30,68)))
model1.add(LSTM(256))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1,'sigmoid'))

model1.summary()


Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_26 (LSTM)              (None, 256)               332800    
                                                                 
 dense_39 (Dense)            (None, 8)                 2056      
                                                                 
 dense_40 (Dense)            (None, 1)                 9         
                                                                 
Total params: 334865 (1.28 MB)
Trainable params: 334865 (1.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [134]:
keras.backend.clear_session()

In [135]:
cp = ModelCheckpoint("src/models/model1", save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=.0001), metrics=[RootMeanSquaredError()])

In [136]:
with tf.device('/GPU:0'):
    model1.fit(X_train,y_train,validation_data=(X_val,y_val), epochs=1000, callbacks=[cp])

Epoch 1/1000
248/250 [============================>.] - ETA: 0s - loss: 0.2465 - root_mean_squared_error: 0.4965INFO:tensorflow:Assets written to: src/models\model1\assets


INFO:tensorflow:Assets written to: src/models\model1\assets


250/250 [==============================] - 7s 24ms/step - loss: 0.2465 - root_mean_squared_error: 0.4965 - val_loss: 0.2494 - val_root_mean_squared_error: 0.4994
Epoch 2/1000
250/250 [==============================] - ETA: 0s - loss: 0.2448 - root_mean_squared_error: 0.4948INFO:tensorflow:Assets written to: src/models\model1\assets


INFO:tensorflow:Assets written to: src/models\model1\assets


250/250 [==============================] - 5s 22ms/step - loss: 0.2448 - root_mean_squared_error: 0.4948 - val_loss: 0.2474 - val_root_mean_squared_error: 0.4974
Epoch 3/1000
250/250 [==============================] - 4s 17ms/step - loss: 0.2445 - root_mean_squared_error: 0.4944 - val_loss: 0.2497 - val_root_mean_squared_error: 0.4997
Epoch 4/1000
250/250 [==============================] - 4s 17ms/step - loss: 0.2444 - root_mean_squared_error: 0.4943 - val_loss: 0.2481 - val_root_mean_squared_error: 0.4981
Epoch 5/1000
250/250 [==============================] - 4s 17ms/step - loss: 0.2447 - root_mean_squared_error: 0.4947 - val_loss: 0.2481 - val_root_mean_squared_error: 0.4981
Epoch 6/1000
250/250 [==============================] - 4s 16ms/step - loss: 0.2441 - root_mean_squared_error: 0.4941 - val_loss: 0.2480 - val_root_mean_squared_error: 0.4980
Epoch 7/1000
250/250 [==============================] - 4s 16ms/step - loss: 0.2442 - root_mean_squared_error: 0.4942 - val_loss: 0.2494 -

In [132]:
model1.predict(X_test)

58/58 [==============================] - 1s 6ms/step


array([[0.27236444],
       [0.7326065 ],
       [0.5513759 ],
       ...,
       [0.638407  ],
       [0.63792914],
       [0.66649187]], dtype=float32)